## 個人課題1
### Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
* リポジトリ名
* 主要な言語
* スターの数
##### 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する

In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
# DBファイルの保存先パス
path = ''

# Google Colabの場合
# path = '/content/'

# DBファイル
db_name = 'exercise-1.db'

# DB接続の確立
# DBファイルが存在しない場合は新規作成される
conn = sqlite3.connect(path + db_name)

# DBへの接続を閉じる
conn.close()

### テーブル作成

In [3]:
path = ''
db_name = 'exercise-1.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # すでにテーブルがある場合は作り直すために削除（初期化用）
    cur.execute('DROP TABLE IF EXISTS exercise_1;')

    # SQL文の作成
    # リポジトリ名(repositories_name), 主要な言語(language), スター数(stars)
    sql = 'CREATE TABLE  exercise_1 (id INTEGER PRIMARY KEY AUTOINCREMENT, repositories_name TEXT, language TEXT, stars TEXT);'

    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

### スクレイピング

In [4]:
url = "https://github.com/orgs/google/repositories"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

#headers = {
#    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#}

def convert_stars_to_int(stars_text):  
    #スター数を数値に変換
    if not stars_text or stars_text == "Unknown" or stars_text == "0": # 空文字、"Unknown"、"0"の場合の処理
        return 0
    if 'k' in stars_text.lower(): # 'k'が含まれる場合の処理
        return int(float(stars_text.lower().replace('k', '')) * 1000) # 'k'を削除して1000倍する
    return int(stars_text.replace(',', '')) # カンマを削除して整数に変換

In [5]:
scraped_data = []  # スクレイピングしたデータを格納するリスト
page = 1 # 開始ページ

print("スクレイピングを開始")

try:
    while True: # 無限ループ開始
        target_url = f"{url}?page={page}" # ページ番号を付与したURL
        print(f"Page {page} を取得中", end="")
        
        res = requests.get(target_url, headers=headers) # HTTPリクエストを送信
        res.raise_for_status()
        
        soup = BeautifulSoup(res.text, 'html.parser')   

        all_li = soup.find_all('li') # ページ内のすべてのliタグを取得
        count_in_page = 0 # 今回のページで取得した件数をカウントする変数
        
        for li in all_li:
            # h3タグの中のaタグを探す
            # リポジトリ名は必ず h3 (見出し) の中にあるため
            h3_tag = li.find('h3')
            if not h3_tag:
                continue # h3がないliはリポジトリ情報ではないのでスキップ
            
            name_tag = h3_tag.find('a')
            if not name_tag:
                continue

            # リポジトリ名取得
            repositories_name = name_tag.text.strip()

            # 言語取得
            language = li.find(class_="ReposListItem-module__Text_4--mkG7R") # 主要な言語のクラス名で検索
            language = language.text.strip() if language else "N/A" # 言語のテキストを取得 


            # スター数取得 (リンク先が stargazers のものを探す)
            star_tag = li.find('a', href=re.compile(r'/stargazers$')) 
            raw_stars = star_tag.text.strip() if star_tag else "0" # スター数のテキストを取得
            stars = convert_stars_to_int(raw_stars) # スター数を数値に変換
            
            scraped_data.append((repositories_name, language, stars)) 
            count_in_page += 1
        
        print(f"-> {count_in_page} 件取得 (累計: {len(scraped_data)} 件)")

        if count_in_page == 0:
            print("データが見つからないため、スクレイピングを終了する。")
            break

        # 次のページがあるか確認
        next_button = soup.find('a', attrs={'rel': 'next'})
        
        if next_button:
            page += 1
            time.sleep(1) 
        else:
            print("最終ページに到達。ループを終了。")
            break

except Exception as e:
    print(f'\nエラーが発生しました (Page {page}):', e)

スクレイピングを開始
Page 1 を取得中-> 30 件取得 (累計: 30 件)
Page 2 を取得中-> 30 件取得 (累計: 60 件)
Page 3 を取得中-> 30 件取得 (累計: 90 件)
Page 4 を取得中-> 30 件取得 (累計: 120 件)
Page 5 を取得中-> 30 件取得 (累計: 150 件)
Page 6 を取得中-> 30 件取得 (累計: 180 件)
Page 7 を取得中-> 30 件取得 (累計: 210 件)
Page 8 を取得中-> 30 件取得 (累計: 240 件)
Page 9 を取得中-> 30 件取得 (累計: 270 件)
Page 10 を取得中-> 30 件取得 (累計: 300 件)
Page 11 を取得中-> 30 件取得 (累計: 330 件)
Page 12 を取得中-> 30 件取得 (累計: 360 件)
Page 13 を取得中-> 30 件取得 (累計: 390 件)
Page 14 を取得中-> 30 件取得 (累計: 420 件)
Page 15 を取得中-> 30 件取得 (累計: 450 件)
Page 16 を取得中-> 30 件取得 (累計: 480 件)
Page 17 を取得中-> 30 件取得 (累計: 510 件)
Page 18 を取得中-> 30 件取得 (累計: 540 件)
Page 19 を取得中-> 30 件取得 (累計: 570 件)
Page 20 を取得中-> 30 件取得 (累計: 600 件)
Page 21 を取得中-> 30 件取得 (累計: 630 件)
Page 22 を取得中-> 30 件取得 (累計: 660 件)
Page 23 を取得中-> 30 件取得 (累計: 690 件)
Page 24 を取得中-> 30 件取得 (累計: 720 件)
Page 25 を取得中-> 30 件取得 (累計: 750 件)
Page 26 を取得中-> 30 件取得 (累計: 780 件)
Page 27 を取得中-> 30 件取得 (累計: 810 件)
Page 28 を取得中-> 30 件取得 (累計: 840 件)
Page 29 を取得中-> 30 件取得 (累計: 870 件)
Page 30 を取得中-> 

### データベースに挿入

In [6]:
path = ''
db_name = 'exercise-1.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # 複数レコードの挿入
    # INSERT INTO テーブル名 VALUES (列に対応したプレースホルダーをカンマ区切りで);
    sql = 'INSERT INTO exercise_1 (repositories_name, language, stars) VALUES (?, ?, ?);'


    # SQL文の実行
    cur.executemany(sql, scraped_data)
    
    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

### データの参照

In [7]:
path = ''
db_name = 'exercise-1.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # データを参照するSQL
    # SELECT * FROM テーブル名;
    # * の部分は，取得したい列の名前を，区切りで指定することもできる．
    sql = "SELECT * FROM exercise_1;"

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

else:
    for row in cur:
        # 行データ(row)はタプルなので，アンパックして列データを取得
        id, repositories_name, language, stars = row
        print(f"ID: {id}, リポジトリ名: {repositories_name}, 言語: {language}, スター数: {stars}")

finally:
    # DBへの接続を閉じる
    conn.close()

ID: 1, リポジトリ名: sedpack, 言語: Python, スター数: 28
ID: 2, リポジトリ名: tunix, 言語: Python, スター数: 1900
ID: 3, リポジトリ名: go-containerregistry, 言語: Go, スター数: 3600
ID: 4, リポジトリ名: bazel-common, 言語: Starlark, スター数: 91
ID: 5, リポジトリ名: XNNPACK, 言語: C, スター数: 2200
ID: 6, リポジトリ名: osv-scalibr, 言語: Go, スター数: 536
ID: 7, リポジトリ名: open-dice, 言語: C++, スター数: 26
ID: 8, リポジトリ名: heir, 言語: C++, スター数: 609
ID: 9, リポジトリ名: dive, 言語: C++, スター数: 17
ID: 10, リポジトリ名: chromium-policy-vulnfeed, 言語: Go, スター数: 7
ID: 11, リポジトリ名: site-kit-wp, 言語: JavaScript, スター数: 1300
ID: 12, リポジトリ名: toucan, 言語: C++, スター数: 48
ID: 13, リポジトリ名: xls, 言語: C++, スター数: 1400
ID: 14, リポジトリ名: nomulus, 言語: Java, スター数: 1800
ID: 15, リポジトリ名: oss-fuzz, 言語: Shell, スター数: 12000
ID: 16, リポジトリ名: docsy, 言語: JavaScript, スター数: 2900
ID: 17, リポジトリ名: gemma.cpp, 言語: C++, スター数: 6600
ID: 18, リポジトリ名: osv-scanner, 言語: Go, スター数: 8100
ID: 19, リポジトリ名: cassowary.dart, 言語: Dart, スター数: 44
ID: 20, リポジトリ名: nsjail, 言語: C++, スター数: 3600
ID: 21, リポジトリ名: zerocopy, 言語: Rust, スター数: 2100
ID: 22, リポジト